In [1]:
import compute_rhino3d.Util
import compute_rhino3d.Grasshopper as gh
import rhino3dm
import json
import pandas as pd
import numpy as np 
import ast
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import time
import datetime
import math
%matplotlib auto

compute_rhino3d.Util.url = "http://localhost:8081/"
#compute_rhino3d.Util.apiKey = ""

Using matplotlib backend: Qt5Agg


In [2]:
import os
import win32api

def kill_exe(name):
    os.system('taskkill /f /t /im '+ name)
    print('kill success')

def start_exe(path):
    win32api.ShellExecute(0, 'open', path, '', '', 1)
    print('start success')

start_exe(r"E:\Software\compute.rhino3d-master\src\bin\Debug\compute\compute.geometry.exe")
time.sleep( 3 )


start success


In [3]:
ROOM_NAMES = [
    'boundary', 'path1', 'path2', 'room1'
]

In [4]:
DIC = {
    'boundary' : [0, 0, 7800, 10200],
    'path1' : [0, 0, 4200, 3600],
    'path2' : [0, 0, 4200, 6600],
    'room1' : [0, 0, 3600, 10200]
}

STATE = pd.DataFrame(DIC)
STATE_ORG = STATE.copy()
STATE, STATE_ORG

(   boundary  path1  path2  room1
 0         0      0      0      0
 1         0      0      0      0
 2      7800   4200   4200   3600
 3     10200   3600   6600  10200,
    boundary  path1  path2  room1
 0         0      0      0      0
 1         0      0      0      0
 2      7800   4200   4200   3600
 3     10200   3600   6600  10200)

In [5]:
#定义动作
STEP = 600
ACTION = [STEP, -STEP, STEP, -STEP, STEP, -STEP, STEP, -STEP]

# environment-同rhino交互

In [6]:
#同rhino交互
def environment(action_index, total_action, action_space = ACTION, in_parm = STATE):

    if action_index == total_action:
        in_parm = in_parm
    else:
        #选择动作        
        room_index = action_index // 8   #取整除整数
        action_index1 = action_index % 8  #取整除余数
        action_index_trans = action_index1 // 2              

        if room_index == 0:
            in_parm['path1'][action_index_trans] = in_parm['path1'][action_index_trans] + action_space[action_index1]
        elif room_index == 1:
            in_parm['path2'][action_index_trans] = in_parm['path2'][action_index_trans] + action_space[action_index1]
        elif room_index == 2:
            in_parm['room1'][action_index_trans] = in_parm['room1'][action_index_trans] + action_space[action_index1]
            
    size_error = (in_parm.loc[2:] <= 0)   #state中的尺寸
    in_parm[size_error] = STEP
    
    #PATH1
    path1_px = gh.DataTree("RH_IN:PATH1_PX")
    path1_px.Append([0], [str(in_parm['path1'][0])])

    path1_py = gh.DataTree("RH_IN:PATH1_PY")
    path1_py.Append([0], [str(in_parm['path1'][1])])

    path1_dx = gh.DataTree("RH_IN:PATH1_DX")
    path1_dx.Append([0], [str(in_parm['path1'][2])])

    path1_dy = gh.DataTree("RH_IN:PATH1_DY")
    path1_dy.Append([0], [str(in_parm['path1'][3])])

    #PATH2
    path2_px = gh.DataTree("RH_IN:PATH2_PX")
    path2_px.Append([0], [str(in_parm['path2'][0])])

    path2_py = gh.DataTree("RH_IN:PATH2_PY")
    path2_py.Append([0], [str(in_parm['path2'][1])])

    path2_dx = gh.DataTree("RH_IN:PATH2_DX")
    path2_dx.Append([0], [str(in_parm['path2'][2])])

    path2_dy = gh.DataTree("RH_IN:PATH2_DY")
    path2_dy.Append([0], [str(in_parm['path2'][3])])

    #ROOM1
    room1_px = gh.DataTree("RH_IN:ROOM1_PX")
    room1_px.Append([0], [str(in_parm['room1'][0])])

    room1_py = gh.DataTree("RH_IN:ROOM1_PY")
    room1_py.Append([0], [str(in_parm['room1'][1])])

    room1_dx = gh.DataTree("RH_IN:ROOM1_DX")
    room1_dx.Append([0], [str(in_parm['room1'][2])])

    room1_dy = gh.DataTree("RH_IN:ROOM1_DY")
    room1_dy.Append([0], [str(in_parm['room1'][3])])


    ####构建输入数据结构####
    trees = [
        path1_px, path1_py, path1_dx, path1_dy,
        path2_px, path2_py, path2_dx, path2_dy,
        room1_px, room1_py, room1_dx, room1_dy
    ]

    #将输入参数赋值到GH模型中，并输出结果
    output = gh.EvaluateDefinition('House_generator_TEST_2021-7-7.gh',trees)

    # decode results
    branch = output['values']

    for item in branch:
        # 读取rhino输出信息
        if item['ParamName'] == 'RH_OUT:SPACIAL_INFO':
            out = item['InnerTree']['{ 0; }']
            #print(out, '\n------')
            spacial_relation = json.loads(out[0]['data'])
            room_info = json.loads(out[1]['data'])
            room_center_info = json.loads(out[2]['data'])
            
        elif item['ParamName'] == 'RH_OUT:PATH_INFO':
            out_path = item['InnerTree']['{ 0; }']
            #print(out_path, '\n---------------')
            path_info = json.loads(out_path[0]['data'])
            
        elif item['ParamName'] == 'RH_OUT:BLANK_AREA':
            out_blank_area = item['InnerTree']['{ 0; }']
            blank_area = json.loads(out_blank_area[0]['data'])

        else:
            pass
    
    output = {
        'spacial_relation': spacial_relation, 
        'room_info': room_info, 
        'room_center_info': room_center_info, 
        'path_info': path_info,
        'blank_area': blank_area
        }
                 
    return output, in_parm, size_error
        

# transform 对rhino输出内容转换

In [7]:
def transform(env):
    
    ##转换为字典
    spacial_relation_dic = ast.literal_eval(env[0]['spacial_relation'])
    room_info_dic = ast.literal_eval(env[0]['room_info'])
    room_centers_dic = ast.literal_eval(env[0]['room_center_info'])
    path_info_dic = ast.literal_eval(env[0]['path_info'])
    blank_area_lis = ast.literal_eval(env[0]['blank_area'])
    
    #房间空间关系  
    ######每个单元格索引，0为空间关系类型，1为相交面积，2为共线长度，-1为同一个房间
    ROOM_NAMES_index = []
    
    for i in ROOM_NAMES:
        ROOM_NAMES_index.append(i + '_') 
    
    RELATION = pd.DataFrame(spacial_relation_dic, index = ROOM_NAMES_index)
    order=ROOM_NAMES
    RELATION_ALL=RELATION[order]  #整理column顺序
    RELATION = RELATION_ALL.drop(columns = ['boundary'], index = ['boundary_'])
    
    #房间长宽
    ROOM_INFO_ALL = pd.DataFrame(room_info_dic, index = ['depth', 'width'])
    ROOM_INFO_ALL = ROOM_INFO_ALL[order]

    #房间几何中心， 索引0 为X，1为Y
    ROOM_INFO_ALL.loc['center'] = room_centers_dic
    ROOM_INFO = ROOM_INFO_ALL.drop(columns = ['boundary'])

    #交通空间信息   在布尔判断中，0代表是，1代表否
    PATH = pd.DataFrame(path_info_dic, index = [0])
    
    output = {
        'relation': RELATION,
        'relation_all': RELATION_ALL,
        'room_info': ROOM_INFO,
        'room_info_all': ROOM_INFO_ALL,
        'path': PATH,
        'blank_area': blank_area_lis,
        'size_error': env[2]
    }

    return output

# 定义目标空间关系

In [8]:
###### 定义目标空间关系邻接矩阵 ######

MATRIX_TARGET = [
    [-1,6,6,6],
    [6,-1,2,2],
    [6,2,-1,2],
    [6,2,2,-1]
]

lis = []
for i in ROOM_NAMES[:4]:
    new_name = i + '_'
    lis.append(new_name)

RELATION_TARGET = pd.DataFrame(MATRIX_TARGET, columns = ROOM_NAMES[:4], index = lis)
RELATION_TARGET

,boundary,path1,path2,room1
boundary_,-1,6,6,6
path1_,6,-1,2,2
path2_,6,2,-1,2
room1_,6,2,2,-1


# 空间关系评价

In [9]:
###### 定义空间关系评价 ######
##值越小越接近要求

class Score_relation:    
    def __init__(self, df, room1, room2, area):
        self.df = df
        self.room1 = room1
        self.room2 = room2
        self.area = area
        self.x_distance = abs(self.df[self.room1]['center'][0] - self.df[self.room2]['center'][0])
        self.y_distance = abs(self.df[self.room1]['center'][1] - self.df[self.room2]['center'][1])
        #print(self.x_distance)
        self.sub_score = 0
        
    def need_seperated(self):  #需要相离1
        x_distance_need = abs(self.df[self.room1]['width'] + self.df[self.room2]['width'])/2   #预期距离
        y_distance_need = abs(self.df[self.room1]['depth'] + self.df[self.room2]['depth'])/2  
        
        x_score = abs(self.x_distance - x_distance_need)
        y_score = abs(self.y_distance - y_distance_need)
        
        if self.x_distance - x_distance_need <=0 and self.y_distance - y_distance_need <=0:
            score = min([x_score, y_score]) + STEP
        elif self.x_distance - x_distance_need <=0 and self.y_distance - y_distance_need >0:
            score = x_score + STEP
        elif self.x_distance - x_distance_need >0 and self.y_distance - y_distance_need <=0:
            score = y_score + STEP
        elif self.x_distance - x_distance_need >0 and self.y_distance - y_distance_need >0:
            score = 0            
        else:
            score = 0

        return score
        
    def need_externally_tangent(self):  #需要外切2
        x_distance_need = abs(self.df[self.room1]['width'] + self.df[self.room2]['width'])/2   #房间几何中心x轴目标距离
        y_distance_need = abs(self.df[self.room1]['depth'] + self.df[self.room2]['depth'])/2   #房间几何中心y轴目标距离
        
        x_score = abs(self.x_distance - x_distance_need)   #将现有几何中心距离同目标距离之差的绝对值，作为score
        y_score = abs(self.y_distance - y_distance_need)
        
        if self.x_distance - x_distance_need >= 0 and self.y_distance - y_distance_need >=0:  
            score = x_score + y_score + STEP
        elif self.x_distance - x_distance_need >= 0 and self.y_distance - y_distance_need < 0:
            score = x_score 
        elif self.x_distance - x_distance_need < 0 and self.y_distance - y_distance_need >= 0:
            score = y_score 
        elif self.x_distance - x_distance_need < 0 and self.y_distance - y_distance_need < 0:
            score = min([x_score, y_score])
            
        return score
        
    def need_intersected(self):    #需要相交3
        room1_width = self.df[self.room1]['width']
        room1_depth = self.df[self.room1]['depth']
        room2_width = self.df[self.room2]['width']
        room2_depth = self.df[self.room2]['depth']
        
        x_distance_need_max = abs(room1_width + room2_width)/2   #预期最大距离
        y_distance_need_max = abs(room1_depth + room2_depth)/2
        
        x_distance_need_min = abs(room1_width - room2_width)/2   #预期最小距离
        y_distance_need_min = abs(room1_depth - room2_depth)/2
        
        x_score = abs(self.x_distance - x_distance_need_max)
        y_score = abs(self.y_distance - y_distance_need_max)
        x_score_min = abs(self.x_distance - x_distance_need_min)
        y_score_min = abs(self.y_distance - y_distance_need_min)
        
        if self.x_distance - x_distance_need_max >= 0 and self.y_distance - y_distance_need_max >= 0:
            score = x_score + y_score + STEP
        elif self.x_distance - x_distance_need_max < 0 and self.y_distance - y_distance_need_max >= 0:
            score = y_score + STEP
        elif self.x_distance - x_distance_need_max >= 0 and self.y_distance - y_distance_need_max < 0:
            score = x_score + STEP
        elif self.x_distance - x_distance_need_max < 0 and self.y_distance - y_distance_need_max < 0:            
            area_room1 = int(room2_width * room1_depth)
            area_room2 = int(room2_width * room2_depth)
            area_min = min([area_room1, area_room2])        
            
            if self.area - area_min >= 0:  #如果面积等于最小房间面积，则判断该状态为内切或包含状态
                score = min([x_score_min, y_score_min]) + STEP
            else:
                score = 0
        return score
    
    def need_internally_tangent(self):    #需要内切4
        room1_width = self.df[self.room1]['width']
        room1_depth = self.df[self.room1]['depth']
        room2_width = self.df[self.room2]['width']
        room2_depth = self.df[self.room2]['depth']
        
        x_distance_need = abs(room1_width - room2_width)/2   #预期最小距离
        y_distance_need = abs(room1_depth - room2_depth)/2
        
        x_score = abs(self.x_distance - x_distance_need)
        y_score = abs(self.y_distance - y_distance_need)

        score_distance = x_score + y_score  #距离
        
        if (room1_width >= room2_width)&(room1_depth >= room2_depth):
            score_size = 0
        elif (room1_width < room2_width)&(room1_depth < room2_depth):
            score_size = 0
        else:
            score_size = min([abs(room1_width - room2_width), abs(room1_depth - room2_depth)]) #边长之差的最小值

        score = score_distance + score_size
        
        return score
        
    def need_contain(self):    #需要包含5
        room1_width = self.df[self.room1]['width']
        room1_depth = self.df[self.room1]['depth']
        room2_width = self.df[self.room2]['width']
        room2_depth = self.df[self.room2]['depth']
        
        x_distance_need = abs(room1_width - room2_width)/2   #预期最小距离
        y_distance_need = abs(room1_depth - room2_depth)/2
        
        x_score = abs(self.x_distance - x_distance_need)
        y_score = abs(self.y_distance - y_distance_need)
        
        if room1_width >= room2_width and room1_depth >= room2_depth:
            score_size = 0
        elif room1_width < room2_width and room1_depth < room2_depth:
            score_size = 0
        else:
            score_size = min([abs(room1_width - room2_width)/4, abs(room1_depth - room2_depth)/4]) #边长之差的最小值
        
        if self.x_distance - x_distance_need >= 0 and self.y_distance - y_distance_need >= 0:               
            score = x_score + y_score + STEP + score_size   
        elif self.x_distance - x_distance_need >= 0 and self.y_distance - y_distance_need < 0:  
            score = x_score + STEP + score_size
        elif self.x_distance - x_distance_need < 0 and self.y_distance - y_distance_need >= 0:  
            score = x_score + STEP + score_size
        elif self.x_distance - x_distance_need < 0 and self.y_distance - y_distance_need < 0:  
            score = score_size        
            
        return score
    
    def need_inside_boundary(self, boundary, room):    #需要相交 + 内切 + 包含6
        boundary_width = self.df[boundary]['width']
        boundary_depth = self.df[boundary]['depth']
        room_width = self.df[room]['width']
        room_depth = self.df[room]['depth']
        
        x_distance = abs(self.df[boundary]['center'][0] - self.df[room]['center'][0])
        y_distance = abs(self.df[boundary]['center'][1] - self.df[room]['center'][1])
        
        x_distance_need = abs(self.df[boundary]['width'] - self.df[room]['width'])/2   #预期最小距离
        y_distance_need = abs(self.df[boundary]['depth'] - self.df[room]['depth'])/2
        
        x_score = abs(x_distance - x_distance_need)
        y_score = abs(y_distance - y_distance_need)
        
        if room_width - boundary_width > 0:
            score_width = room_width - boundary_width
        else:
            score_width = 0
        if room_depth - boundary_depth > 0:
            score_depth = room_depth - boundary_depth
        else:
            score_depth = 0
        
        if x_distance - x_distance_need > 0 and y_distance - y_distance_need > 0:
            score_distance = x_score + y_score
        elif x_distance - x_distance_need > 0 and y_distance - y_distance_need <= 0:
            score_distance = x_score
        elif x_distance - x_distance_need <= 0 and y_distance - y_distance_need > 0:
            score_distance = y_score
        elif x_distance - x_distance_need <= 0 and y_distance - y_distance_need <= 0:
            score_distance = 0
        
        score = score_distance + score_width + score_depth
            
        return score


# 房间尺寸评价

score_relation = Score_relation(ROOM_INFO, 'living', 'hallway')
score_relation.need_externally_tangent()
score_relation.need_seperated()
score_relation.need_intersected()
score_relation.need_internally_tangent()
score_relation.need_contain()

In [10]:
###### 尺寸评价 #####
class Score_size:    
    def __init__(self, ROOM_INFO):
        self.ROOM_INFO = ROOM_INFO

    #房间尺寸
    def score_room_size(self, room_name, depth_min, depth_max, width_min, width_max):
        depth = self.ROOM_INFO[room_name]['depth']
        width = self.ROOM_INFO[room_name]['width']
        
        if depth > depth_max or depth < depth_min:
            score_depth = abs(depth - depth_max)
        elif depth <= depth_max and depth >= depth_min:
            score_depth = 0

        if width > width_max or width < width_min:
            score_width = abs(width - width_max)
        elif width <= width_max and width >= width_min:
            score_width = 0

        score = score_depth + score_width
        return score
    

# 交通形态评价

In [11]:
class Score_path_form:
    def __init__(self, path):
        self.path = path
        self.vertical_size = self.path['vertical_minsize'][0]
        self.horizontal_size = self.path['horizontal_minsize'][0]
        self.vertice_number = self.path['vertice_number'][0]
        
        if self.path['area'][0] == None:
            self.area = 0
        else:
            self.area = float(self.path['area'][0])        

    def minsize(self):

        SIZE_MAX = 1200
        
        if self.area == 0 or self.vertical_size == 1000000000 or self.horizontal_size == 1000000000:
            score = 0
        else:
            if self.vertical_size >= 600 and self.horizontal_size >= 600:
                score = -SIZE_MAX
            elif self.vertical_size < 600 and self.horizontal_size >= 600:
                score = self.vertical_size + 600 - SIZE_MAX
            elif self.vertical_size >= 600 and self.horizontal_size < 600:
                score = self.horizontal_size + 600 - SIZE_MAX
            elif self.vertical_size < 600 and self.horizontal_size < 600:
                score = self.horizontal_size + self.vertical_size - SIZE_MAX
        
        return score
    
    def path_area(self):
        area_m = self.area/1000000
        AREA_MAX = 40
        
        if self.area == 0 or self.vertical_size == 1000000000 or self.horizontal_size == 1000000000:
            score = 0
        else:
            if area_m > AREA_MAX:
                score = 0
            else:
                score = (area_m - AREA_MAX) * STEP
        
        return score         
    
    def vertice(self):

        MAX_NUMBER = 40
        
        if self.area == 0 or self.vertical_size == 1000000000 or self.horizontal_size == 1000000000:
            score = 0
        else:
            if self.vertice_number > MAX_NUMBER:
                score = 0
            else:
                score = (self.vertice_number - MAX_NUMBER) * STEP
                
        return score       
        

# 其它得分

In [12]:
class Score_others():

    def score_area(self, area):
        #score_blank = area[0]/(STEP*STEP)
        #score_out = area[1]/(STEP*STEP)
        score_blank = pow(area[0]/(STEP*STEP), 0.5)    #开平方根
        score_out = pow(area[1]/(STEP*STEP), 0.5)
        return score_blank, score_out
    


# reward_calculation总得分计算

b = Score_size(ROOM_INFO, RELATION)
c = b.score_room_size(room_name = 'room1', depth_min = 9000, depth_max = 9000, width_min = 2700, width_max = 3000)
d = score_path_boundary(PATH, BOUNDARY)
d

In [13]:
########## 计算总得分 ##########

def reward_curve_exp(x):
    score = pow(1.04,x+6)*4 - pow(1.04,6)*4
    if score > 1:
        score = 1
    else:
        pass
    return score

def reward_curve_log(x):
    score = np.log(x + 10)*10 - np.log(0 + 10)*10
    return score

def reward_calculation(trans):
    RELATION = trans['relation']
    RELATION_ALL = trans['relation_all']
    ROOM_INFO = trans['room_info']
    ROOM_INFO_ALL = trans['room_info_all']
    PATH = trans['path']
    SIZE_ERROR = trans['size_error']
    BLANK_AREA = trans['blank_area']
    #print(type(BLANK_AREA))

    relation_score = 0

    #房间空间关系得分
    for i in RELATION_TARGET.columns:
        for j in RELATION_TARGET.index:
            if RELATION_TARGET[i][j] == -1:
                pass
            else:            
                if RELATION_TARGET[i][j] == RELATION_ALL[i][j][0]:
                    pass
                else:
                    area = RELATION_ALL[i][j][1]
                    score_relation = Score_relation(ROOM_INFO_ALL, i, j[:-1], area = area)
                    
                    if RELATION_TARGET[i][j] == 1:  #相离
                        subscore = score_relation.need_seperated()/STEP

                    elif RELATION_TARGET[i][j] == 2:   #外切
                        subscore= score_relation.need_externally_tangent()/STEP

                    elif RELATION_TARGET[i][j] == 3:   #相交
                        subscore = score_relation.need_intersected()/STEP

                    elif RELATION_TARGET[i][j] == 4:   #内切
                        subscore = score_relation.need_internally_tangent()/STEP

                    elif RELATION_TARGET[i][j] == 5:   #包含
                        subscore = score_relation.need_contain()/STEP
                        
                    elif RELATION_TARGET[i][j] == 6:   #非相离,inside boundary
                        if i == 'boundary':
                            subscore = score_relation.need_inside_boundary(boundary = i, room = j[:-1])/STEP
                        if j == 'boundary_':
                            subscore = score_relation.need_inside_boundary(boundary = j[:-1], room = i)/STEP

                    relation_score = relation_score + subscore

    #尺寸得分
    #score_size = Score_size(ROOM_INFO_ALL)
    
    #for item in ROOM_INFO.columns:
        #if item == 'path1':
            #subscore_size = score_size.score_room_size(room_name = item , depth_min = 10200, depth_max = 7800, width_min = 0, width_max = 0)
            #subscore_size1 = reward_curve_log(subscore_size/STEP)
        #elif item == 'room1':
            #subscore1 = score_size.score_room_size(room_name = item , depth_min = 1200, depth_max = 3600, width_min = 1200, width_max = 3600)
        #elif item == 'path2':
            #subscore1 = score_size.score_room_size(room_name = item , depth_min = 1200, depth_max = 3600, width_min = 1200, width_max = 3600)
        
        #total_score = total_score + subscore_size1

    #其它得分
    others = Score_others()
    subscore_area = others.score_area(area = BLANK_AREA)
    
    ###最后汇总
    score_blank = subscore_area[0]
    score_out = subscore_area[1]
       
    total_score = relation_score + score_blank + score_out

    return total_score, relation_score, score_blank, score_out


# 定义DQN网络

In [14]:
#####定义 DQN #####
#超参数
BATCH_SIZE = 32
LR = 0.00002
EPSILON = 0.8 #随机选取的概率，如果概率小于这个随机数，就采取greedy的行为
GAMMA = 0.9
TARGET_REPLACE_ITER = 100  #target_net 更新参数频率
MEMORY_CAPACITY = 30000
START_LEARN = 1000
TOTAL_EPISODE = 1000
EPISODE_STEPS = 100   #每回合内步数
N_STATES = 12
N_ACTIONS = 24


In [15]:
class Net(nn.Module):
    def __init__(self, feature =N_STATES, hidden0 = 480, hidden = 480, output=N_ACTIONS, func = 'tanh'):
        super(Net, self).__init__()
        self.func = func
        
        self.input = nn.Linear(feature, hidden0)
        self.input.weight.data.normal_(0, 0.1)   # initialization        
        self.fc1 = nn.Linear(hidden0, hidden)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc2.weight.data.normal_(0, 0.1)   # initialization
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc3.weight.data.normal_(0, 0.1)   # initialization
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc4.weight.data.normal_(0, 0.1)   # initialization
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc5.weight.data.normal_(0, 0.1)   # initialization
        self.fc6 = nn.Linear(hidden, hidden)
        self.fc6.weight.data.normal_(0, 0.1)   # initialization
        self.fc7 = nn.Linear(hidden, hidden0)
        self.fc7.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(hidden0, output)
        self.out.weight.data.normal_(0, 0.1)   # initialization
        
        if self.func == 'relu' :
            self.activate = nn.ReLU() 
        elif self.func == 'lrelu':
            self.activate = nn.LeakyReLU(negative_slope = 0.01,inplace=True)
        elif self.func == 'tanh':
            self.activate = nn.Tanh()

    def forward(self, x):
        x = self.input(x)
        x = self.activate(x)
        x = self.fc1(x)
        x = self.activate(x)
        x = self.fc2(x)
        x = self.activate(x)
        x = self.fc3(x)
        x = self.activate(x)
        x = self.fc4(x)
        x = self.activate(x)
        x = self.fc5(x)
        x = self.activate(x)
        x = self.fc6(x)
        x = self.activate(x)
        x = self.fc7(x)
        x = self.activate(x)
        out = self.out(x)        
        return out

In [16]:
class DQN(object):
    def __init__(self):        
        self.DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.eval_net, self.target_net = Net().to(self.DEVICE), Net().to(self.DEVICE)
        self.eval_net.load_state_dict(torch.load('1_72369_DQN_24action.pkl'))
        self.target_net.load_state_dict(torch.load('1_72369_DQN_24action.pkl'))
        
        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 1 + 1))     # state*2 + rewards + action_index
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        #self.optimizer = torch.optim.SGD(self.eval_net.parameters(), lr= LR, momentum=0.7)
        self.loss_func = nn.MSELoss() #优化器和损失函数        
    
    # 接收环境中的观测值，并采取动作
    def choose_action(self, x):        
        x = torch.unsqueeze(torch.FloatTensor(x), 0).to(self.DEVICE)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x).to('cpu')
            action= torch.max(actions_value, 1)[1].data.numpy()  #返回动作最大值索引
            action = action[0] # return the argmax index
            
        else:   # random
            action = np.random.randint(0, N_ACTIONS)  #返回随机索引
            
        return action
    
        
    #记忆库，存储之前的记忆，学习之前的记忆库里的东西
    def store_transition(self, s, a, r, s_):
      
        transition = np.hstack((s, [a, r], s_)) #水平堆叠
        #print('transation: ',transition)
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1     
        
    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1       
        
        if self.memory_counter <= MEMORY_CAPACITY:
            sample_index = np.random.choice(self.memory_counter, BATCH_SIZE)
            b_memory = self.memory[sample_index, :]    
        else:            
            sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
            b_memory = self.memory[sample_index, :]        
        
        b_s = torch.FloatTensor(b_memory[:, :N_STATES]).to(self.DEVICE)
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)).to(self.DEVICE)
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]).to(self.DEVICE)
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:]).to(self.DEVICE)
        
        #print('b_s:', b_s, 'b_a:', b_a, 'b_r:', b_r)
        
        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # q_next.max(1)[0]返回每行最大值的数值    
        
        loss = self.loss_func(q_eval, q_target)
        #print('损失函数：',loss)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()        
                
        #for name, parms in self.eval_net.named_parameters():
            #print('-->name:', name, '-->grad_requirs:',parms.requires_grad, '\n -->grad_value:',parms.grad)
        
        #print('q_eval: ', q_eval, 'q_target:',q_next.max(1)[0].view(BATCH_SIZE, 1), 'b_r:', b_r, 'loss: ', loss)


# 绘图

In [17]:
def drawing_graphs(episode, iteration, state, x, y, x1, y1):
    #绘制动态图表
    plt.figure(12, figsize = (12,6))
    plt.title(episode)
    if (dqn.memory_counter == START_LEARN + 1) or (episode % 10 == 0 and iteration == 1):
        plt.clf()
        global ax1
        ax1 = plt.subplot(1,2,1)
        #plt.xlim(0, EPISODE_STEPS)
        #plt.ylim(-15, 25)
    else:
        pass
    
    if dqn.memory_counter == START_LEARN + 1:
        pass
    else:
        pass
    
    plt.xlim(0, EPISODE_STEPS)
    plt.xticks(fontproperties = 'Times New Roman', size = 15)
    plt.yticks(fontproperties = 'Times New Roman', size = 15)
  
    xx = x[iteration-1:iteration+1]
    yy = y[iteration-1:iteration+1]
    ax1.plot(xx, yy, color='steelblue', alpha = 0.4)  # 绘制曲线
    plt.grid(True,linestyle = '-.')  
    
    if iteration == EPISODE_STEPS-1:
        ax1.text(x[iteration],y[iteration],i_episode,ha='center',va='center',fontsize=12)
        print(' Ep: ', i_episode,'| Ep_reward: ',ep_r)
    
    ####图2
    #if dqn.memory_counter == START_LEARN + 1:
    if (dqn.memory_counter == START_LEARN + 1) or (episode % 10 == 0 and iteration == 1):
        global ax2
        ax2 = plt.subplot(1,2,2)
    else:
        pass

    #if episode % 10 == 0 and iteration == 1:
        #plt.cla()
        #plt.xlim((0, EPISODE_STEPS))
        #plt.ylim(-1, 10)
    #else:
        #pass
        
    plt.xlim((0, EPISODE_STEPS))
    plt.xticks(fontproperties = 'Times New Roman', size = 15)
    plt.yticks(fontproperties = 'Times New Roman', size = 15)

    xx1 = x1[iteration-1:iteration+1]
    yy1 = y1[iteration-1:iteration+1]
    ax2.plot(xx1, yy1, color='steelblue', alpha = 0.4)  # 绘制曲线
    plt.grid(True,linestyle = '-.')
    plt.pause(0.00005)
    
    if iteration == EPISODE_STEPS-1:
        ax2.text(x1[iteration],y1[iteration],i_episode,ha='center',va='bottom',fontsize=12)
        print(' Ep: ', i_episode,'| Ep_reward: ',ep_r) 

    plt.figure(13, figsize = (10,10)) 
    if dqn.memory_counter == START_LEARN + 1:
        global ax3
        ax3 = plt.subplot(1,1,1)
    else:
        pass

    plt.cla()
    plt.xlim((-20000, 25000))
    plt.ylim(-20000, 25000)
    plt.xticks(fontproperties = 'Times New Roman', size = 15)
    plt.yticks(fontproperties = 'Times New Roman', size = 15)
    plt.grid(True,linestyle = '-.')
    plt.gca().add_patch(plt.Rectangle(xy=(state.loc[0,'path1'],state.loc[1,'path1']),width=state.loc[2,'path1'],height=state.loc[3,'path1'],alpha = 0.4))
    plt.gca().add_patch(plt.Rectangle(xy=(state.loc[0,'boundary'],state.loc[1,'boundary']),width=state.loc[2,'boundary'],height=state.loc[3,'boundary'],alpha = 0.4))
    plt.gca().add_patch(plt.Rectangle(xy=(state.loc[0,'path2'],state.loc[1,'path2']),width=state.loc[2,'path2'],height=state.loc[3,'path2'],alpha = 0.4, color = 'r'))
    plt.gca().add_patch(plt.Rectangle(xy=(state.loc[0,'room1'],state.loc[1,'room1']),width=state.loc[2,'room1'],height=state.loc[3,'room1'],alpha = 0.4, color = 'y'))   
    
    plt.pause(0.00005)


# training

In [ ]:
def sigmoid(x):
    out = ((1/(1 + np.exp(-x))) - 0.5)*2
    return out

dqn = DQN()
old_best_result = 0

print('\nCollection experience...')

for i_episode in range(TOTAL_EPISODE):

    STATE = STATE_ORG.copy()  #回合结束，复位STATE

    #随机初始化房间
    #random_location = np.random.randint(-1,12,[2,3])*STEP
    #random_size = np.random.randint(1,12,[2,3])*STEP
    #random = np.vstack((random_location, random_size))    
    
    #STATE.iloc[:,1:] = random.tolist()
    
    start_time = datetime.datetime.now()
    ep_r = 0
    reward_recorder = []
    best_result_counter = 0
    
    gragh_x = list(np.linspace(0,EPISODE_STEPS-1,EPISODE_STEPS))
    gragh_y = [0]
    gragh_x1 = list(np.linspace(0,EPISODE_STEPS-1,EPISODE_STEPS))
    gragh_y1 = [0]
    
    plt.ion()
    for iterate in range(EPISODE_STEPS):   
        #r_recorder = []
        
        if iterate == 0:       
            out_env = environment(action_index = N_ACTIONS, total_action = N_ACTIONS, action_space = ACTION, in_parm = STATE.copy())
            out_trans = transform(env = out_env)
            reward = reward_calculation(trans = out_trans)
            reward_recorder.append(reward)
            print(STATE)            
        
        else:
            s = np.array(STATE[ROOM_NAMES[1:]])
            s_in = np.hstack(s)

            #选择动作
            action = dqn.choose_action(x = s_in)

            #观察环境，得到新的状态
            out_env = environment(action_index = action, total_action = N_ACTIONS, action_space = ACTION, in_parm = STATE.copy())
            STATE = pd.DataFrame(out_env[1])  #更新STATE
            out_trans = transform(env = out_env)
            print(STATE)

            ######### 计算奖励 ###########
            #原奖励函数为结果越差得分越高，取负数使得结果越好r越高
            reward = reward_calculation(trans = out_trans)
            reward_recorder.append(reward)
            
            if reward[0] == 0:
                best_result_counter += 1                
            
            r_relation = reward_recorder[-2][1] - reward_recorder[-1][1]
            r_blank = (reward_recorder[-2][2] - reward_recorder[-1][2])*5
            r_out = (reward_recorder[-2][3] - reward_recorder[-1][3])*5
            
            reward0 = (r_relation + r_blank + r_out)/2
            
            print('r_relation: ',r_relation, 'r_blank: ',r_blank, 'r_out: ', r_out, 'r_all: ', reward0)
                       
            if True in out_env[2].values:   #如果生成矩形尺寸<0,则给予惩罚
                r1 = -1
            else:
                r1 = 0
            
            if reward0 < 0:       #r为负数时，惩罚更严厉，在原有基础上减0.5
                r2 = -1
            else:
                r2 = 0
                
            if reward[0] == 0:   #达到全局最优时，增加奖励，在原有基础上加0.5
                r3 = 2
            else:
                r3 = 0
                
            #r_recorder.append(r0)
            #counter = set(r_recorder)
            #if len(set(r_recorder)) == 1 and r_recorder[0] == 0 and r_now != 0:   #如果连续6次奖励为0，则施加惩罚，减分
                #r4 = -0.3
            #else:
                #r4 = 0 
            #r_recorder = r_recorder[-6:]
            
            r = reward0 + r1 + r2 + r3 #+ r4
            
            #其它状态提取
            a = action
            s_ = np.array(STATE[ROOM_NAMES[1:]])
            s_in_ = np.hstack(s_)

            #存储信息
            dqn.store_transition(s = s_in, a = a, r = r, s_ = s_in_)
            
            ep_r += r   #计算总奖励
            gragh_y.append(ep_r)  #更新绘图表的y轴参数
            gragh_y1.append(reward_recorder[-1][0])
            
            if dqn.memory_counter % 1 == 0:
                print('memory_counter:', dqn.memory_counter, '| r:', r, ' | score_now: ', reward_recorder[-1][0])
                        
            if dqn.memory_counter > START_LEARN:
                dqn.learn()
                drawing_graphs(episode=i_episode, iteration=iterate, state=STATE, x=gragh_x, y=gragh_y, x1=gragh_x1, y1=gragh_y1)
                
            #重启虚拟服务器进程
            if dqn.memory_counter % 100 == 0:
                kill_exe('compute.geometry.exe')
                time.sleep( 2 )
                start_exe(r"E:\Software\compute.rhino3d-master\src\bin\Debug\compute\compute.geometry.exe")
                time.sleep( 10 )
                                           
    plt.ioff()
    plt.show()
    
    #探索率设置
    if dqn.memory_counter < START_LEARN:
        EPSILON = 0.8
    elif dqn.memory_counter > START_LEARN and dqn.memory_counter < MEMORY_CAPACITY:
        EPSILON = 0.9
    else:
        EPSILON = 0.95
    
    if best_result_counter > old_best_result or i_episode % 200 == 0:
        model_name = 'dqn_model\\' + str(best_result_counter) + '_' + str(dqn.memory_counter) + '_DQN_24action.pkl'
        torch.save(dqn.eval_net.state_dict(), model_name)
        #torch.save(dqn.eval_net, model_name)
        print('model saved sucessfully! model_name: ', model_name)        
        
    end_time = datetime.datetime.now()
    print('time: ', str((end_time - start_time).seconds))

